In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

In [2]:
from fredapi import Fred
fred = Fred('Your_api')

In [3]:
#Resesion
#"GS10"   # 10-year constant maturity Treasury yield
#"TB3MS"  # 3-month Tbill rate
#"UNRATE" # unemployment rate
#"NROU"   # estimate of natural rate of unemployment from U.S. Congressional Budget Office


In [4]:
srty = pd.DataFrame()
srty['USREC'] = pd.DataFrame(fred.get_series('USREC'))
srty['FEDFUNDS'] = pd.DataFrame(fred.get_series('FEDFUNDS'))
srty['date'] = srty.index
srty.dropna(inplace=True)
srty['USREC'] = srty['USREC'].map({1: 'Yes', 0: 'No'})

In [5]:
fig = px.line(x=srty['date'], y=srty['FEDFUNDS'])



# Highlight areas according to the even_type
current = ''
changes = []

for ind in srty.index:
    if current != srty['USREC'][ind]:
        current = srty['USREC'][ind]
        changes.append([srty['date'][ind],srty['USREC'][ind]])


color_dict= {'Yes':'#d9330d','No':'#e5ecf6',}

for i, val in enumerate(changes):
    if (i+1) == len(changes):
            changes_last_date = changes[-1][0]
            changes_last_state = changes[-1][1]
            last_date= srty['date'].iloc[-1]
            fig.add_vrect(
                          x0=changes_last_date, x1=last_date,
                          fillcolor=color_dict[changes_last_state], opacity=0.2,
                          layer="below", line_width=0)

    else:
          

        if changes[i][1] =='Yes':
            fig.add_vrect(
                        x0=changes[i][0], x1=changes[i+1][0],
                        fillcolor=color_dict['Yes'], opacity=0.2,
                        layer="below", line_width=0)


fig.update_layout(xaxis_title="Date", yaxis_title="Values", title='Federal Funds Effective Rate',xaxis = {'showgrid': False}, yaxis = {'showgrid': True})

fig.show()

In [6]:
#s1 = pd.DataFrame(fred.get_series('TB3MS'))
#s2 = pd.DataFrame(fred.get_series('GS10'))
#s = s2-s1
s = pd.DataFrame(fred.get_series('T10Y2Y'))

In [7]:

# sample data
df = pd.DataFrame()
df['USREC'] = srty['USREC']
df['y'] = s
df['ma1'] = s
df['ma2'] = 0
df=df.dropna()
df1 = df.copy()

# split data into chunks where averages cross each other
df['label'] = np.where(df['ma1']>df['ma2'], 1, 0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()
df = df.groupby('group')
dfs = []
for name, data in df:
    dfs.append(data)

# custom function to set fill color
def fillcol(label):
    if label >= 1:
        return 'rgba(250,0,0,0.4)'
    else:
        return 'rgba(0,250,0,0.4)'

fig = go.Figure()

for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df.ma1,
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df.ma2,
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = fillcol(df['label'].iloc[0])))

# include averages
fig.add_traces(go.Scatter(x=df1.index, y = df1.ma1,
                          line = dict(color = 'blue', width=1)))

fig.add_traces(go.Scatter(x=df1.index, y = df1.ma2,
                          line = dict(color = 'red', width=1)))

# include main time-series
fig.add_traces(go.Scatter(x=df1.index, y = df1.y,
                          line = dict(color = 'black', width=2)))


fig.update_layout(
        title='10-Year Treasury Constant Maturity Minus 2-Year Treasury Constant Maturity',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Percent',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1 # gap between bars of the same location coordinate.
    )


# Highlight areas according to the even_type
current = ''
changes = []

for ind in srty.index:
    if current != srty['USREC'][ind]:
        current = srty['USREC'][ind]
        changes.append([srty['date'][ind],srty['USREC'][ind]])


color_dict= {'Yes':'#d9330d','No':'#e5ecf6',}

for i, val in enumerate(changes):
    if (i+1) == len(changes):
            changes_last_date = changes[-1][0]
            changes_last_state = changes[-1][1]
            last_date= srty['date'].iloc[-1]
            fig.add_vrect(
                          x0=changes_last_date, x1=last_date,
                          fillcolor=color_dict[changes_last_state], opacity=0.2,
                          layer="below", line_width=0)

    else:
          

        if changes[i][1] =='Yes':
            fig.add_vrect(
                        x0=changes[i][0], x1=changes[i+1][0],
                        fillcolor=color_dict['Yes'], opacity=0.2,
                        layer="below", line_width=0)


fig.update_layout(showlegend=False)
fig.show()

In [8]:
UNRATE = pd.DataFrame(fred.get_series('UNRATE'))
NROU = pd.DataFrame(fred.get_series('NROU'))

# sample data
df = pd.DataFrame()
df['USREC'] = srty['USREC']
df['y'] = UNRATE.iloc[:,0]
df['ma1'] = NROU.iloc[:,0]
df['ma2'] = UNRATE.iloc[:,0]
df=df.dropna()
df1 = df.copy()

# split data into chunks where averages cross each other
df['label'] = np.where(df['ma1']>df['ma2'], 1, 0)
df['group'] = df['label'].ne(df['label'].shift()).cumsum()
df = df.groupby('group')
dfs = []
for name, data in df:
    dfs.append(data)

# custom function to set fill color
def fillcol(label):
    if label >= 1:
        return 'rgba(0,250,0,0.4)'
    else:
        return 'rgba(250,0,0,0.4)'

fig = go.Figure()

for df in dfs:
    fig.add_traces(go.Scatter(x=df.index, y = df.ma1,
                              line = dict(color='rgba(0,0,0,0)')))
    
    fig.add_traces(go.Scatter(x=df.index, y = df.ma2,
                              line = dict(color='rgba(0,0,0,0)'),
                              fill='tonexty', 
                              fillcolor = fillcol(df['label'].iloc[0])))

# include averages
fig.add_traces(go.Scatter(x=df1.index, y = df1.ma1,
                          line = dict(color = 'blue', width=1)))

fig.add_traces(go.Scatter(x=df1.index, y = df1.ma2,
                          line = dict(color = 'red', width=1)))

# include main time-series
fig.add_traces(go.Scatter(x=df1.index, y = df1.y,
                          line = dict(color = 'black', width=2)))

fig.update_layout(
        title='U.S. UnemploymentRate vs Natural Rate of Unemployment',
        xaxis_tickfont_size=14,
        yaxis=dict(
            title='Percent',
            titlefont_size=16,
            tickfont_size=14,
        ),
        legend=dict(
            x=0,
            y=1.0,
            bgcolor='rgba(255, 255, 255, 0)',
            bordercolor='rgba(255, 255, 255, 0)'
        ),
        barmode='group',
        bargap=0.15, # gap between bars of adjacent location coordinates.
        bargroupgap=0.1 # gap between bars of the same location coordinate.
    )

# Highlight areas according to the even_type
current = ''
changes = []

for ind in srty.index:
    if current != srty['USREC'][ind]:
        current = srty['USREC'][ind]
        changes.append([srty['date'][ind],srty['USREC'][ind]])


color_dict= {'Yes':'#d9330d','No':'#e5ecf6',}

for i, val in enumerate(changes):
    if (i+1) == len(changes):
            changes_last_date = changes[-1][0]
            changes_last_state = changes[-1][1]
            last_date= srty['date'].iloc[-1]
            fig.add_vrect(
                          x0=changes_last_date, x1=last_date,
                          fillcolor=color_dict[changes_last_state], opacity=0.2,
                          layer="below", line_width=0)

    else:
          

        if changes[i][1] =='Yes':
            fig.add_vrect(
                        x0=changes[i][0], x1=changes[i+1][0],
                        fillcolor=color_dict['Yes'], opacity=0.2,
                        layer="below", line_width=0)


fig.update_layout(showlegend=False)
fig.show()

In [9]:
vix = pd.DataFrame()
vix['VIXCLS'] = pd.DataFrame(fred.get_series('VIXCLS'))
vix['date'] = vix.index
vix['USREC'] = srty['USREC']
vix['USREC'] = vix['date'].map(lambda x: x.replace(day=1))\
                                         .map(srty['USREC'])

vix = vix.fillna('No')
fig = px.line(x=vix['date'], y=vix['VIXCLS'])



# Highlight areas according to the even_type
current = ''
changes = []

for ind in vix.index:
    if current != vix['USREC'][ind]:
        current = vix['USREC'][ind]
        changes.append([vix['date'][ind],vix['USREC'][ind]])


color_dict= {'Yes':'#d9330d','No':'#e5ecf6',}

for i, val in enumerate(changes):
    if (i+1) == len(changes):
            changes_last_date = changes[-1][0]
            changes_last_state = changes[-1][1]
            last_date= vix['date'].iloc[-1]
            fig.add_vrect(
                          x0=changes_last_date, x1=last_date,
                          fillcolor=color_dict[changes_last_state], opacity=0.2,
                          layer="below", line_width=0)

    else:
          

        if changes[i][1] =='Yes':
            fig.add_vrect(
                        x0=changes[i][0], x1=changes[i+1][0],
                        fillcolor=color_dict['Yes'], opacity=0.2,
                        layer="below", line_width=0)


fig.update_layout(xaxis_title="Date", yaxis_title="Values", title='CBOE Volatility Index',xaxis = {'showgrid': False}, yaxis = {'showgrid': True})

fig.show()

In [10]:
vix = pd.DataFrame()
vix['SAHMREALTIME'] = pd.DataFrame(fred.get_series('SAHMREALTIME'))
vix['date'] = vix.index
vix['USREC'] = srty['USREC']
vix['USREC'] = vix['date'].map(lambda x: x.replace(day=1))\
                                         .map(srty['USREC'])

vix = vix.fillna('No')
fig = px.line(x=vix['date'], y=vix['SAHMREALTIME'])



# Highlight areas according to the even_type
current = ''
changes = []

for ind in vix.index:
    if current != vix['USREC'][ind]:
        current = vix['USREC'][ind]
        changes.append([vix['date'][ind],vix['USREC'][ind]])


color_dict= {'Yes':'#d9330d','No':'#e5ecf6',}

for i, val in enumerate(changes):
    if (i+1) == len(changes):
            changes_last_date = changes[-1][0]
            changes_last_state = changes[-1][1]
            last_date= vix['date'].iloc[-1]
            fig.add_vrect(
                          x0=changes_last_date, x1=last_date,
                          fillcolor=color_dict[changes_last_state], opacity=0.2,
                          layer="below", line_width=0)

    else:
          

        if changes[i][1] =='Yes':
            fig.add_vrect(
                        x0=changes[i][0], x1=changes[i+1][0],
                        fillcolor=color_dict['Yes'], opacity=0.2,
                        layer="below", line_width=0)


fig.update_layout(xaxis_title="Date", yaxis_title="Values", title='Sahm Rule Recession Indicator',xaxis = {'showgrid': False}, yaxis = {'showgrid': True})

fig.show()